<a href="https://colab.research.google.com/github/ajaym416/TreeleafAIchallenge/blob/main/resnet_18_for_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import os
import time
import numpy as np
from torchvision import datasets , models
from torch.utils.data import Dataset , DataLoader
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import logging
from tqdm.notebook import tqdm
import torchvision
import torch.nn as nn


In [ ]:
#for viewing our outputs and progress in tensorboard
Writer = SummaryWriter("/content/drive/MyDrive/tree_ai/Output")


In [ ]:
# %%python
import logging
log = logging.getLogger('tree_logger')
logging.basicConfig(
    level=logging.INFO # allow INFO level messages to pass through the logger
    )

In [ ]:
!unzip /content/drive/MyDrive/liveness_detection.zip -d /content/

In [ ]:
#make a list to contain the path of all the images to be used fot training
fake_images_folder_path ="/content/fake_images"
genuine_images_folder_path = "/content/genuine_images"
log.info(f"the number of fakes images is:{len(os.listdir(fake_images_folder_path))}" )
log.info(f"the number of genuine images is:{len(os.listdir(genuine_images_folder_path))}")
folders=[fake_images_folder_path , genuine_images_folder_path]
train_images_path=[]
for folder_path in folders:
  for images in os.listdir(folder_path):
    if images[-3:] !='jpg':
      #since all our images are in jpg , we are checking if there are any other files  other than the jpg images
      log.info(images)
    else:
      file_path = os.path.join(folder_path,images)
    train_images_path.append(file_path)

INFO:tree_logger:the number of fakes images is:15093
INFO:tree_logger:the number of genuine images is:14120


In [ ]:
#Different Transformations for the data augmentation 
import torchvision.transforms as T
def get_train_transform():
  return (T.Compose([
                     T.RandomHorizontalFlip(p=0.5),
                     T.RandomRotation(15),
                     T.RandomCrop(204,),
                     T.ToTensor(),
                     T.ColorJitter(brightness=(0.5,1.5), contrast=(0.5,1.5), saturation=(0.5,1.5), hue=(-0.1,0.1)),
                    #  T.Normalize(mean=[0.4732, 0.3933, 0.3759], std=[0.2000, 0.1905, 0.1839])
                     T.Normalize(mean=[0.4652, 0.3855, 0.3666],std=[0.2024, 0.1921, 0.1849])

                    ]))
def get_val_transform():
  return (T.Compose([T.ToTensor(),
                     ]))

In [ ]:
class FaceDataset(Dataset):
    def __init__(self,img_size = 224 ,train=True,test=False ,val_stride =10 ,sort_by_random =True,train_images =train_images_path,transforms=get_train_transform()):
        self.img_size =img_size
        self.train = train
        self.val_stride = val_stride
        self.sort_by_random= sort_by_random
        self.train_images=train_images.copy()
        self.transforms=transforms
        self.test=test
        if self.sort_by_random:
          np.random.shuffle(self.train_images)
        if self.train:
          del self.train_images[::val_stride]
        elif self.test:
          self.train_images = self.train_images[::val_stride].copy()
          del self.train_images[::2]
        else:
          self.train_images = self.train_images[::val_stride].copy()
          self.train_images=self.train_images[::2]


    def __len__(self):
        return len(self.train_images)
    def __getitem__(self ,idx):
        image_path = self.train_images[idx]
        img = Image.open(image_path)
        img = img.resize((self.img_size, self.img_size))
        if (os.path.split(self.train_images[idx])[-2][-11:])=='fake_images':
          label =np.eye(2)[0]
          label = torch.tensor(label, dtype = torch.float32)
        else:
          label =np.eye(2)[1]
          label = torch.tensor(label, dtype = torch.float32)
                  
        img = self.transforms(img)
        return img, label


In [ ]:
#Getting the training, validation and test_set
train_data = FaceDataset()
val_data = FaceDataset(train=False,transforms=get_val_transform())
test_data = FaceDataset(train=False,test=True,transforms=get_val_transform())
log.info(f"train: {len(train_data)},val: {len(val_data)},test: {len(test_data)}")

INFO:tree_logger:train: 26291,val: 1461,test: 1461


In [ ]:
#check if the GPU is availabe
use_cuda =torch.cuda.is_available()
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
log.info(f"Training on device {device}.")

INFO:tree_logger:Training on device cuda.


In [ ]:
#Data loader to load the data to the model by getting the data from dataset
train_dl = torch.utils.data.DataLoader( 
    train_data,
    batch_size=64,
    shuffle=True,
    num_workers=8,
)
val_dl=torch.utils.data.DataLoader( 
    val_data,
    batch_size=64,
    shuffle=True,
    num_workers=8,
)
test_dl =torch.utils.data.DataLoader( 
    test_data,
    batch_size=64,
    shuffle=True,
    num_workers=8,
)

In [ ]:
#Display sample images used for training
fig = plt.figure(figsize=(30, 30))
columns = 4
rows = 8
images, labels = next(iter(train_dl))
# ax enables access to manipulate each of subplots
ax = []
for i in range(columns*rows):
  ax.append(fig.add_subplot(rows,columns,i+1))

for i,img in enumerate(images):
  if i>31:
    break
  ax[i].imshow(img.permute(1,2,0))
  if labels[i][0]==0:
    ax[i].set_title("genuine")
  else:
    ax[i].set_title("Fake" )

plt.show()  # finally, render the plot
image_grid = torchvision.utils.make_grid(images)
Writer.add_image('Face',image_grid)

In [ ]:
# #To get the the mean and standard deviation of our dataset
# train_data = FaceDataset()
# loader = DataLoader(
#     train_data,
#     batch_size=64,
#     num_workers=8,
#     shuffle=False
# )

# mean = 0.
# std = 0.
# nb_samples = 0.
# for data in loader:
#   data ,labels =data
#   batch_samples = data.size(0)
#   data = data.view(batch_samples, data.size(1), -1)
#   mean += data.mean(2).sum(0)
#   std += data.std(2).sum(0)
#   nb_samples += batch_samples

# mean /= nb_samples
# std /= nb_samples
# print(mean, std)

In [ ]:
#get the pretrained model for transfer learning
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(512, 2),
    nn.Sigmoid()
)

In [ ]:
# def Accuracy(preds, labels):
#   preds =[[1 if preds[i][j]>= 0.5 else 0 for j in range(preds.shape[1])] for i in range(preds.shape[0])]
#   total =len(preds)
#   correct = np.sum([(preds[i]==np.array(torch.Tensor.tolist(labels)).astype(int)[i])[0] for i in range(len(labels))])
#   return correct/total
def Accuracy(preds, labels):
  total =len(preds)
  correct=0
  value, preds = torch.max( preds, 1)
  correct = torch.sum(preds == torch.argmax(labels,1)).item()
  return correct/total

In [ ]:
def train_one_epoch(train_data_loader,epoch):
    
    ### Local Parameters
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()
    
    ###Iterating over data loader
    for i,(images, labels) in tqdm(enumerate(train_data_loader)):
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)

        #Reseting Gradients
        optimizer.zero_grad()
        
        #Forward
        preds = model(images)
        
        #Calculating Loss
        _loss = criterion(preds, labels)
        loss = _loss.item()
        epoch_loss.append(loss)
        
        #Calculating Accuracy
        acc = Accuracy(preds, labels)
        epoch_acc.append(acc)
        
        #Backward
        _loss.backward()
        optimizer.step()
    Writer.add_scalar("Training Loss" ,loss,epoch)
    Writer.add_scalar("Training Accuracy" ,acc,epoch)

    
    ###Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    ###Storing results to logs
    train_logs["loss"].append(epoch_loss)
    train_logs["accuracy"].append(epoch_acc)
    train_logs["time"].append(total_time)
    
    return epoch_loss, epoch_acc, total_time

In [ ]:
def val_one_epoch(val_data_loader,epoch, best_val_acc):
    
    ### Local Parameters
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()
    
    ###Iterating over data loader
    for i,(images, labels) in enumerate(val_data_loader):
        
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward
        preds = model(images)
        
        #Calculating Loss
        _loss = criterion(preds, labels)
        loss = _loss.item()
        epoch_loss.append(loss)
        
        #Calculating Accuracy
        acc = Accuracy(preds, labels)
        epoch_acc.append(acc)
    Writer.add_scalar("Validation Loss" ,loss,epoch)
    Writer.add_scalar("Validation Accuracy" ,acc,epoch)
    
    ###Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    ###Storing results to logs
    val_logs["loss"].append(epoch_loss)
    val_logs["accuracy"].append(epoch_acc)
    val_logs["time"].append(total_time)
    
    ###Saving best model
    if epoch_acc > best_val_acc:
        best_val_acc = epoch_acc
        torch.save(model.state_dict(),"/content/drive/MyDrive/liveness_detection_final_submission.pth")
        # torch.save(model.state_dict(),"resnet18_best_face.pth")
        
    return epoch_loss, epoch_acc, total_time, best_val_acc

In [ ]:
def test_one_epoch(test_data_loader,epoch):
    
    ### Local Parameters
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()
    
    ###Iterating over data loader
    for i,(images, labels) in enumerate(test_data_loader):
        
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward
        preds = model(images)
        
        #Calculating Loss
        _loss = criterion(preds, labels)
        loss = _loss.item()
        epoch_loss.append(loss)
        
        #Calculating Accuracy
        acc = Accuracy(preds, labels)
        epoch_acc.append(acc)
    Writer.add_scalar("Test Loss" ,loss,epoch)
    Writer.add_scalar("Test Accuracy" ,acc,epoch)
    
    ###Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    ###Storing results to logs
    test_logs["loss"].append(epoch_loss)
    test_logs["accuracy"].append(epoch_acc)
    test_logs["time"].append(total_time)
        
    return epoch_loss, epoch_acc, total_time

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
# Learning Rate Scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)
#Loss Function
criterion = nn.BCELoss()
# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}
test_logs={"loss" : [], "accuracy" : [], "time" : []}
# Loading model to device
model.to(device)
# No of epochs 
epochs = 50

In [ ]:
best_val_acc = 0
for epoch in range(epochs):
    ###Training
    loss, acc, _time = train_one_epoch(train_dl,epoch)
    log.info(f"Training : Epoch={epoch+1} , Loss={round(loss, 4)} , Acc= {round(acc, 4)} , Time{round(_time, 4)} ")
    
    ###Validation
    loss,acc,_time,best_val_acc = val_one_epoch(val_dl,epoch, best_val_acc)
    log.info(f"Validation : Epoch={epoch+1} , Loss={round(loss, 4)} , Acc= {round(acc, 4)} , Time{round(_time, 4)} ")

    ###Testing
    loss,acc,_time = test_one_epoch(val_dl,epoch)
    log.info(f"Testing : Epoch={epoch+1} , Loss={round(loss, 4)} , Acc= {round(acc, 4)} , Time{round(_time, 4)} ")

In [ ]:
with open('/content/drive/MyDrive/tree_ai/train_logs.txt', 'w') as f:
    print(train_logs, file=f)
with open('/content/drive/MyDrive/tree_ai/val_logs.txt', 'w') as f:
    print(train_logs, file=f)
with open('/content/drive/MyDrive/tree_ai/test_logs.txt', 'w') as f:
    print(train_logs, file=f)


In [ ]:
train_logs

In [ ]:
val_logs

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/tree_ai/Output